# Tavsiye Sistemleri

<b>Tavsiye sistemleri (recommendation system - RS)</b>, size ne alacağınızı, nerede yemek yiyeceğinizi ve hatta kiminle arkadaş olmanız gerektiğini söyleyerek web/ürün deneyiminizi kişiselleştirmek için kullanılırlar. İnsanların zevkleri farklı olmasına rağmen genellikle belli kalıpları takip eder. Kişiler, beğendikleri diğer şeylere benzer şeyleri alma eğilimindedir ve yakın oldukları diğer insanlarla benzer zevklere sahip olma eğilimindedir. Tavsiye sistemleri, sevebileceğiniz başka şeyleri tahmin etmek için bu kalıpları yakalamaya çalışır. E-ticaret, sosyal medya, video ve çevrimiçi haber platformları, müşterilerinin ürünleri daha verimli bir şekilde seçmelerine yardımcı olmak için kendi tavsiye sistemlerini aktif olarak kullanmaktadır, bu sayede rekabet avantajını artırırlar.

Bu yazıda en yaygın tavsiye sistemleri türlerinden olan <b>*içerik tabanlı filtreleme (content-based filtering)*</b> ve <b>*işbirlikçi filtreleme (collaborative filtering - CF)*</b> yöntemlerinden bahsedilecektir. İşbirlikçi filtreleme, kullanıcıların öğelere karşı tutumlarının bilgisine dayanarak öneriler üretir, yani öğeleri önermek için "kalabalığın bilgeliğini" kullanır. Buna karşılık, içerik tabanlı tavsiye sistemleri, öğelerin özelliklerine odaklanır ve aralarındaki benzerliğe dayalı olarak size önerilerde bulunur.


Genel olarak, İşbirlikçi filtreleme, öneri motorlarının en önemli parçasıdır. Algoritma, özellik öğrenimini kendi başına yapabilmek gibi çok ilginç bir özelliğe sahiptir. Bu sayede hangi özellikleri kullanacağını algoritma kendi belirler. İşbirlikçi filtreleme, <b>*Bellek Tabanlı İşbirliğine Dayalı Filtreleme (Memory-Based Collaborative Filtering)*</b> ve <b>*Model Tabanlı İşbirliğine Dayalı Filtreleme (Model-Based Collaborative filtering)*</b> olarak ikiye ayrılabilir. Bu uygulamada, <b>*tekil değer ayrıştırması (singular value decomposition - SVD)*</b> kullanarak Model Tabanlı CF ve kosinüs benzerliği hesabı ile Bellek Tabanlı CF uygulanacaktır.

Tavsiye sistemlerini uygularken ve test ederken en yaygın kullanılan veri kümelerinden biri olan MovieLens veri kümesini kullanacağız. Veri seti 943 kullanıcının 1.682 film için yaptığı 100.000 film derecelendirmesi içermektedir. Movielens veri kümesi klasörünü Jupyter Notebook dizininize eklemelisiniz.

<b>Veri setini aşağıdaki bağlantıdan indirebilirsiniz:<b>
- [http://files.grouplens.org/datasets/movielens/ml-100k.zip](http://files.grouplens.org/datasets/movielens/ml-100k.zip)
    
Veri setinin kısa açıklamasına [buradan](http://files.grouplens.org/datasets/movielens/ml-100k-README.txt) erişebilirsiniz.

In [1]:
import numpy as np
import pandas as pd

In [2]:
header = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('ml-100k/u.data', sep='\t', names=header)

In [3]:
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]

print('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_items))

Number of users = 943 | Number of movies = 1682


Veri setinde, 943 adet unique kullanıcı ve 1682 adette unique film bulunuyor.  

In [5]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.25)

Veri setini, train_data (%75) ve test_data (%25) olarak rastgele bir şekilde `train_test_split` fonksiyonu ile ikiye bölüyoruz. 

### Bellek Tabanlı İşbirliğine Dayalı Filtreleme

Bellek Tabanlı İşbirlikçi Filtreleme yaklaşımları temelde iki ana bölüme ayrılabilir: <b>*kullanıcı-öğe (user-item) filtreleme*</b> ve <b>*öğe-öğe(item-item) filtreleme*</b>. Kullanıcı-öğe filtrelemesi belirli bir kullanıcıyı alır, benzerliğin derecesine dayalı olarak hedef kullanıcıyla benzer nitelikteki kullanıcıları tespit eder ve benzer kullanıcıların beğendikleri öğeleri önerir. Öte yandan, öğe-öğe filtrelemesi bir öğeyi alır, o öğeyi beğenen kullanıcıları tespit eder ve bu kullanıcıların veya benzer kullanıcıların aynı zamanda beğendiği diğer öğeleri de tespit eder. İşin sonunda, girdi olarak aldığı öğelerden hedef kullanıcılara öğeleri önerir.

+ **Kullanıcı-Öğe İşbirlikçi Filtreleme:** "Sizin gibi kullanıcılar aynı zamanda şunları da beğendi..."
+ **Öğe-Öğe İşbirlikçi Filtreleme:** "Bu öğeyi beğenen kullanıcılar aynı zamanda şunları da beğendi..."

Her iki durumda da, tüm veri setini kullanarak bir kullanıcı-öğe matrisi oluşturmamız gerekir. Veriyi test ve eğitim olarak böldüğümüzden dolayı, iki adet `[943 × 1682]` matris oluşturmamız gerekecektir. Eğitim matrisi toplam derecelendirmelerin %75'ini içerirken, test matrisi derecelendirmelerin %25'ini içerir.

Örnek bir kullanıcı-öğe matrisi:

<img src='https://buomsoo-kim.github.io/data/images/2020-09-25/1.png' width='600' height='500'> 

Kullanıcı-öğe matrisini oluşturulduktan sonra benzerliği hesaplanır ve bir benzerlik matrisi oluşturulur.

Öğe-Öğe İşbirlikçi Filtreleme'de öğeler arasındaki benzerlik değerleri, her iki öğeyi de derecelendiren tüm kullanıcıların gözlemlenmesi yoluyla belirlenir.

Kullanıcı-Öğe İşbirlikçi Filtreleme'de kullanıcılar arasındaki benzerlik değerleri, her iki kullanıcının da derecelendirdiği tüm öğelerin gözlemlenmesi yoluyla belirlenir.

Tavsiye sistemlerinde yaygın olarak kullanılan [benzerlik ölçüsü](https://github.com/enesmanan/turkce-kitaplar/tree/main/Projelerle%20Yapay%20Zeka/Benzerlik_Algoritmalar%C4%B1) <b>*kosinüs benzerliğidir.*</b> Derecelendirmeler n boyutlu uzayda vektörler olarak temsil edilir ve benzerliğin derecesi vektörler arasındaki açıya göre hesaplanır. a ve m kullanıcıları için kosinüs benzerliği, aşağıdaki formül kullanılarak hesaplanabilir; burada kullanıcı vektörü $U_k$ ile kullanıcı vektörü $U_a$'nın nokta çarpımı alınır ve vektörlerin Öklid uzunluklarının çarpımına bölünür. 

<p><span class="math display"><span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" id="MathJax-Element-7-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot; display=&quot;block&quot;><msubsup><mi>s</mi><mi>u</mi><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi>c</mi><mi>o</mi><mi>s</mi></mrow></msubsup><mo stretchy=&quot;false&quot;>(</mo><msub><mi>u</mi><mi>k</mi></msub><mo>,</mo><msub><mi>u</mi><mi>a</mi></msub><mo stretchy=&quot;false&quot;>)</mo><mo>=</mo><mfrac><mrow><msub><mi>u</mi><mi>k</mi></msub><mo>&amp;#x22C5;</mo><msub><mi>u</mi><mi>a</mi></msub></mrow><mrow><mrow><mo symmetric=&quot;true&quot;>&amp;#x2016;</mo><msub><mi>u</mi><mi>k</mi></msub><mo symmetric=&quot;true&quot;>&amp;#x2016;</mo></mrow><mrow><mo symmetric=&quot;true&quot;>&amp;#x2016;</mo><msub><mi>u</mi><mi>a</mi></msub><mo symmetric=&quot;true&quot;>&amp;#x2016;</mo></mrow></mrow></mfrac><mo>=</mo><mfrac><mrow><mo>&amp;#x2211;</mo><msub><mi>x</mi><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi>k</mi><mo>,</mo><mi>m</mi></mrow></msub><msub><mi>x</mi><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi>a</mi><mo>,</mo><mi>m</mi></mrow></msub></mrow><msqrt><mo>&amp;#x2211;</mo><msubsup><mi>x</mi><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi>k</mi><mo>,</mo><mi>m</mi></mrow><mn>2</mn></msubsup><mo>&amp;#x2211;</mo><msubsup><mi>x</mi><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi>a</mi><mo>,</mo><mi>m</mi></mrow><mn>2</mn></msubsup></msqrt></mfrac></math>" role="presentation" style="text-align: center; position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-23" style="width: 20.148em; display: inline-block;"><span style="display: inline-block; position: relative; width: 19.566em; height: 0px; font-size: 103%;"><span style="position: absolute; clip: rect(0.633em, 1019.57em, 4.177em, -999.998em); top: -2.279em; left: 0em;"><span class="mrow" id="MathJax-Span-24"><span class="msubsup" id="MathJax-Span-25"><span style="display: inline-block; position: relative; width: 1.507em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.44em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-26" style="font-family: MathJax_Math-italic;">s</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.546em, 1001.07em, 4.129em, -999.998em); top: -4.318em; left: 0.488em;"><span class="texatom" id="MathJax-Span-27"><span class="mrow" id="MathJax-Span-28"><span class="mi" id="MathJax-Span-29" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mi" id="MathJax-Span-30" style="font-size: 70.7%; font-family: MathJax_Math-italic;">o</span><span class="mi" id="MathJax-Span-31" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.546em, 1000.49em, 4.129em, -999.998em); top: -3.833em; left: 0.488em;"><span class="mi" id="MathJax-Span-32" style="font-size: 70.7%; font-family: MathJax_Math-italic;">u</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-33" style="font-family: MathJax_Main;">(</span><span class="msubsup" id="MathJax-Span-34"><span style="display: inline-block; position: relative; width: 1.022em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-35" style="font-family: MathJax_Math-italic;">u</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.585em;"><span class="mi" id="MathJax-Span-36" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-37" style="font-family: MathJax_Main;">,</span><span class="msubsup" id="MathJax-Span-38" style="padding-left: 0.148em;"><span style="display: inline-block; position: relative; width: 1.022em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-39" style="font-family: MathJax_Math-italic;">u</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.585em;"><span class="mi" id="MathJax-Span-40" style="font-size: 70.7%; font-family: MathJax_Math-italic;">a</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-41" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-42" style="font-family: MathJax_Main; padding-left: 0.294em;">=</span><span class="mfrac" id="MathJax-Span-43" style="padding-left: 0.294em;"><span style="display: inline-block; position: relative; width: 4.323em; height: 0px; margin-right: 0.1em; margin-left: 0.1em;"><span style="position: absolute; clip: rect(3.4em, 1002.82em, 4.274em, -999.998em); top: -4.658em; left: 50%; margin-left: -1.405em;"><span class="mrow" id="MathJax-Span-44"><span class="msubsup" id="MathJax-Span-45"><span style="display: inline-block; position: relative; width: 1.022em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-46" style="font-family: MathJax_Math-italic;">u</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.585em;"><span class="mi" id="MathJax-Span-47" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-48" style="font-family: MathJax_Main; padding-left: 0.245em;">⋅</span><span class="msubsup" id="MathJax-Span-49" style="padding-left: 0.245em;"><span style="display: inline-block; position: relative; width: 1.022em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-50" style="font-family: MathJax_Math-italic;">u</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.585em;"><span class="mi" id="MathJax-Span-51" style="font-size: 70.7%; font-family: MathJax_Math-italic;">a</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.109em, 1004.03em, 4.371em, -999.998em); top: -3.25em; left: 50%; margin-left: -2.085em;"><span class="mrow" id="MathJax-Span-52"><span class="mrow" id="MathJax-Span-53"><span class="mo" id="MathJax-Span-54" style="font-family: MathJax_Main;">∥</span><span class="msubsup" id="MathJax-Span-55"><span style="display: inline-block; position: relative; width: 1.022em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-56" style="font-family: MathJax_Math-italic;">u</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.585em;"><span class="mi" id="MathJax-Span-57" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-58" style="font-family: MathJax_Main;">∥</span></span><span class="mrow" id="MathJax-Span-59" style="padding-left: 0.148em;"><span class="mo" id="MathJax-Span-60" style="font-family: MathJax_Main;">∥</span><span class="msubsup" id="MathJax-Span-61"><span style="display: inline-block; position: relative; width: 1.022em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-62" style="font-family: MathJax_Math-italic;">u</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.585em;"><span class="mi" id="MathJax-Span-63" style="font-size: 70.7%; font-family: MathJax_Math-italic;">a</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-64" style="font-family: MathJax_Main;">∥</span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(0.876em, 1004.32em, 1.216em, -999.998em); top: -1.308em; left: 0em;"><span style="display: inline-block; overflow: hidden; vertical-align: 0em; border-top: 1.3px solid; width: 4.323em; height: 0px;"></span><span style="display: inline-block; width: 0px; height: 1.07em;"></span></span></span></span><span class="mo" id="MathJax-Span-65" style="font-family: MathJax_Main; padding-left: 0.294em;">=</span><span class="mfrac" id="MathJax-Span-66" style="padding-left: 0.294em;"><span style="display: inline-block; position: relative; width: 7.381em; height: 0px; margin-right: 0.1em; margin-left: 0.1em;"><span style="position: absolute; clip: rect(3.109em, 1004.86em, 4.42em, -999.998em); top: -4.706em; left: 50%; margin-left: -2.425em;"><span class="mrow" id="MathJax-Span-67"><span class="mo" id="MathJax-Span-68" style="font-family: MathJax_Size1; vertical-align: 0em;">∑</span><span class="msubsup" id="MathJax-Span-69" style="padding-left: 0.148em;"><span style="display: inline-block; position: relative; width: 1.847em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-70" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.585em;"><span class="texatom" id="MathJax-Span-71"><span class="mrow" id="MathJax-Span-72"><span class="mi" id="MathJax-Span-73" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-74" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-75" style="font-size: 70.7%; font-family: MathJax_Math-italic;">m</span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-76"><span style="display: inline-block; position: relative; width: 1.847em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-77" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.585em;"><span class="texatom" id="MathJax-Span-78"><span class="mrow" id="MathJax-Span-79"><span class="mi" id="MathJax-Span-80" style="font-size: 70.7%; font-family: MathJax_Math-italic;">a</span><span class="mo" id="MathJax-Span-81" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-82" style="font-size: 70.7%; font-family: MathJax_Math-italic;">m</span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(2.769em, 1007.24em, 4.857em, -999.998em); top: -2.91em; left: 50%; margin-left: -3.638em;"><span class="msqrt" id="MathJax-Span-83"><span style="display: inline-block; position: relative; width: 7.235em; height: 0px;"><span style="position: absolute; clip: rect(3.012em, 1006.22em, 4.614em, -999.998em); top: -3.978em; left: 1.022em;"><span class="mrow" id="MathJax-Span-84"><span class="mo" id="MathJax-Span-85" style="font-family: MathJax_Size1; vertical-align: 0em;">∑</span><span class="msubsup" id="MathJax-Span-86" style="padding-left: 0.148em;"><span style="display: inline-block; position: relative; width: 1.847em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-87" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.352em, 1000.44em, 4.129em, -999.998em); top: -4.318em; left: 0.585em;"><span class="mn" id="MathJax-Span-88" style="font-size: 70.7%; font-family: MathJax_Main;">2</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.352em, 1001.26em, 4.274em, -999.998em); top: -3.638em; left: 0.585em;"><span class="texatom" id="MathJax-Span-89"><span class="mrow" id="MathJax-Span-90"><span class="mi" id="MathJax-Span-91" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-92" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-93" style="font-size: 70.7%; font-family: MathJax_Math-italic;">m</span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-94" style="font-family: MathJax_Size1; vertical-align: 0em; padding-left: 0.148em;">∑</span><span class="msubsup" id="MathJax-Span-95" style="padding-left: 0.148em;"><span style="display: inline-block; position: relative; width: 1.847em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-96" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.352em, 1000.44em, 4.129em, -999.998em); top: -4.318em; left: 0.585em;"><span class="mn" id="MathJax-Span-97" style="font-size: 70.7%; font-family: MathJax_Main;">2</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.546em, 1001.26em, 4.274em, -999.998em); top: -3.833em; left: 0.585em;"><span class="texatom" id="MathJax-Span-98"><span class="mrow" id="MathJax-Span-99"><span class="mi" id="MathJax-Span-100" style="font-size: 70.7%; font-family: MathJax_Math-italic;">a</span><span class="mo" id="MathJax-Span-101" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-102" style="font-size: 70.7%; font-family: MathJax_Math-italic;">m</span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.546em, 1006.26em, 3.886em, -999.998em); top: -4.755em; left: 1.022em;"><span style="display: inline-block; position: relative; width: 6.265em; height: 0px;"><span style="position: absolute; font-family: MathJax_Main; top: -3.978em; left: -0.095em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; font-family: MathJax_Main; top: -3.978em; left: 5.585em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 0.391em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 0.925em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 1.459em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 1.944em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 2.478em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 2.964em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 3.498em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 4.032em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 4.517em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 5.051em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(2.672em, 1001.02em, 4.76em, -999.998em); top: -3.93em; left: 0em;"><span style="font-family: MathJax_Size2;">√</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(0.876em, 1007.38em, 1.216em, -999.998em); top: -1.308em; left: 0em;"><span style="display: inline-block; overflow: hidden; vertical-align: 0em; border-top: 1.3px solid; width: 7.381em; height: 0px;"></span><span style="display: inline-block; width: 0px; height: 1.07em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.284em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.848em; border-left: 0px solid; width: 0px; height: 3.452em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML" display="block"><msubsup><mi>s</mi><mi>u</mi><mrow class="MJX-TeXAtom-ORD"><mi>c</mi><mi>o</mi><mi>s</mi></mrow></msubsup><mo stretchy="false">(</mo><msub><mi>u</mi><mi>k</mi></msub><mo>,</mo><msub><mi>u</mi><mi>a</mi></msub><mo stretchy="false">)</mo><mo>=</mo><mfrac><mrow><msub><mi>u</mi><mi>k</mi></msub><mo>⋅</mo><msub><mi>u</mi><mi>a</mi></msub></mrow><mrow><mrow><mo symmetric="true">‖</mo><msub><mi>u</mi><mi>k</mi></msub><mo symmetric="true">‖</mo></mrow><mrow><mo symmetric="true">‖</mo><msub><mi>u</mi><mi>a</mi></msub><mo symmetric="true">‖</mo></mrow></mrow></mfrac><mo>=</mo><mfrac><mrow><mo>∑</mo><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>m</mi></mrow></msub><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>a</mi><mo>,</mo><mi>m</mi></mrow></msub></mrow><msqrt><mo>∑</mo><msubsup><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>m</mi></mrow><mn>2</mn></msubsup><mo>∑</mo><msubsup><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>a</mi><mo>,</mo><mi>m</mi></mrow><mn>2</mn></msubsup></msqrt></mfrac></math></span></span></div><script type="math/tex; mode=display" id="MathJax-Element-7">
s_u^{cos}(u_k,u_a)
 = \frac{ u_k \cdot u_a }{ \left \| u_k \right \| \left \| u_a \right \| }
 = \frac{ \sum x_{k,m}x_{a,m} }{ \sqrt{\sum x_{k,m}^2\sum x_{a,m}^2} }
</script></span></p>

m ve b öğeleri arasındaki benzerliği hesaplamak için aşağıdaki formül kullanılır:

<p><span class="math display"><span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" id="MathJax-Element-10-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot; display=&quot;block&quot;><msubsup><mi>s</mi><mi>u</mi><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi>c</mi><mi>o</mi><mi>s</mi></mrow></msubsup><mo stretchy=&quot;false&quot;>(</mo><msub><mi>i</mi><mi>m</mi></msub><mo>,</mo><msub><mi>i</mi><mi>b</mi></msub><mo stretchy=&quot;false&quot;>)</mo><mo>=</mo><mfrac><mrow><msub><mi>i</mi><mi>m</mi></msub><mo>&amp;#x22C5;</mo><msub><mi>i</mi><mi>b</mi></msub></mrow><mrow><mrow><mo symmetric=&quot;true&quot;>&amp;#x2016;</mo><msub><mi>i</mi><mi>m</mi></msub><mo symmetric=&quot;true&quot;>&amp;#x2016;</mo></mrow><mrow><mo symmetric=&quot;true&quot;>&amp;#x2016;</mo><msub><mi>i</mi><mi>b</mi></msub><mo symmetric=&quot;true&quot;>&amp;#x2016;</mo></mrow></mrow></mfrac><mo>=</mo><mfrac><mrow><mo>&amp;#x2211;</mo><msub><mi>x</mi><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi>a</mi><mo>,</mo><mi>m</mi></mrow></msub><msub><mi>x</mi><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi>a</mi><mo>,</mo><mi>b</mi></mrow></msub></mrow><msqrt><mo>&amp;#x2211;</mo><msubsup><mi>x</mi><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi>a</mi><mo>,</mo><mi>m</mi></mrow><mn>2</mn></msubsup><mo>&amp;#x2211;</mo><msubsup><mi>x</mi><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi>a</mi><mo>,</mo><mi>b</mi></mrow><mn>2</mn></msubsup></msqrt></mfrac></math>" role="presentation" style="text-align: center; position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-109" style="width: 19.226em; display: inline-block;"><span style="display: inline-block; position: relative; width: 18.643em; height: 0px; font-size: 103%;"><span style="position: absolute; clip: rect(0.633em, 1018.64em, 4.177em, -999.998em); top: -2.279em; left: 0em;"><span class="mrow" id="MathJax-Span-110"><span class="msubsup" id="MathJax-Span-111"><span style="display: inline-block; position: relative; width: 1.507em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.44em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-112" style="font-family: MathJax_Math-italic;">s</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.546em, 1001.07em, 4.129em, -999.998em); top: -4.318em; left: 0.488em;"><span class="texatom" id="MathJax-Span-113"><span class="mrow" id="MathJax-Span-114"><span class="mi" id="MathJax-Span-115" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span class="mi" id="MathJax-Span-116" style="font-size: 70.7%; font-family: MathJax_Math-italic;">o</span><span class="mi" id="MathJax-Span-117" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.546em, 1000.49em, 4.129em, -999.998em); top: -3.833em; left: 0.488em;"><span class="mi" id="MathJax-Span-118" style="font-size: 70.7%; font-family: MathJax_Math-italic;">u</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-119" style="font-family: MathJax_Main;">(</span><span class="msubsup" id="MathJax-Span-120"><span style="display: inline-block; position: relative; width: 1.022em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.29em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-121" style="font-family: MathJax_Math-italic;">i</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.342em;"><span class="mi" id="MathJax-Span-122" style="font-size: 70.7%; font-family: MathJax_Math-italic;">m</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-123" style="font-family: MathJax_Main;">,</span><span class="msubsup" id="MathJax-Span-124" style="padding-left: 0.148em;"><span style="display: inline-block; position: relative; width: 0.731em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.29em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-125" style="font-family: MathJax_Math-italic;">i</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.342em;"><span class="mi" id="MathJax-Span-126" style="font-size: 70.7%; font-family: MathJax_Math-italic;">b</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-127" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-128" style="font-family: MathJax_Main; padding-left: 0.294em;">=</span><span class="mfrac" id="MathJax-Span-129" style="padding-left: 0.294em;"><span style="display: inline-block; position: relative; width: 4.032em; height: 0px; margin-right: 0.1em; margin-left: 0.1em;"><span style="position: absolute; clip: rect(3.158em, 1002.53em, 4.274em, -999.998em); top: -4.658em; left: 50%; margin-left: -1.26em;"><span class="mrow" id="MathJax-Span-130"><span class="msubsup" id="MathJax-Span-131"><span style="display: inline-block; position: relative; width: 1.022em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.29em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-132" style="font-family: MathJax_Math-italic;">i</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.342em;"><span class="mi" id="MathJax-Span-133" style="font-size: 70.7%; font-family: MathJax_Math-italic;">m</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-134" style="font-family: MathJax_Main; padding-left: 0.245em;">⋅</span><span class="msubsup" id="MathJax-Span-135" style="padding-left: 0.245em;"><span style="display: inline-block; position: relative; width: 0.731em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.29em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-136" style="font-family: MathJax_Math-italic;">i</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.342em;"><span class="mi" id="MathJax-Span-137" style="font-size: 70.7%; font-family: MathJax_Math-italic;">b</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.109em, 1003.79em, 4.371em, -999.998em); top: -3.25em; left: 50%; margin-left: -1.939em;"><span class="mrow" id="MathJax-Span-138"><span class="mrow" id="MathJax-Span-139"><span class="mo" id="MathJax-Span-140" style="font-family: MathJax_Main;">∥</span><span class="msubsup" id="MathJax-Span-141"><span style="display: inline-block; position: relative; width: 1.022em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.29em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-142" style="font-family: MathJax_Math-italic;">i</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.342em;"><span class="mi" id="MathJax-Span-143" style="font-size: 70.7%; font-family: MathJax_Math-italic;">m</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-144" style="font-family: MathJax_Main;">∥</span></span><span class="mrow" id="MathJax-Span-145" style="padding-left: 0.148em;"><span class="mo" id="MathJax-Span-146" style="font-family: MathJax_Main;">∥</span><span class="msubsup" id="MathJax-Span-147"><span style="display: inline-block; position: relative; width: 0.731em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.29em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-148" style="font-family: MathJax_Math-italic;">i</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.342em;"><span class="mi" id="MathJax-Span-149" style="font-size: 70.7%; font-family: MathJax_Math-italic;">b</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-150" style="font-family: MathJax_Main;">∥</span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(0.876em, 1004.03em, 1.216em, -999.998em); top: -1.308em; left: 0em;"><span style="display: inline-block; overflow: hidden; vertical-align: 0em; border-top: 1.3px solid; width: 4.032em; height: 0px;"></span><span style="display: inline-block; width: 0px; height: 1.07em;"></span></span></span></span><span class="mo" id="MathJax-Span-151" style="font-family: MathJax_Main; padding-left: 0.294em;">=</span><span class="mfrac" id="MathJax-Span-152" style="padding-left: 0.294em;"><span style="display: inline-block; position: relative; width: 7.041em; height: 0px; margin-right: 0.1em; margin-left: 0.1em;"><span style="position: absolute; clip: rect(3.109em, 1004.57em, 4.42em, -999.998em); top: -4.706em; left: 50%; margin-left: -2.279em;"><span class="mrow" id="MathJax-Span-153"><span class="mo" id="MathJax-Span-154" style="font-family: MathJax_Size1; vertical-align: 0em;">∑</span><span class="msubsup" id="MathJax-Span-155" style="padding-left: 0.148em;"><span style="display: inline-block; position: relative; width: 1.847em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-156" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.585em;"><span class="texatom" id="MathJax-Span-157"><span class="mrow" id="MathJax-Span-158"><span class="mi" id="MathJax-Span-159" style="font-size: 70.7%; font-family: MathJax_Math-italic;">a</span><span class="mo" id="MathJax-Span-160" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-161" style="font-size: 70.7%; font-family: MathJax_Math-italic;">m</span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-162"><span style="display: inline-block; position: relative; width: 1.507em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-163" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.585em;"><span class="texatom" id="MathJax-Span-164"><span class="mrow" id="MathJax-Span-165"><span class="mi" id="MathJax-Span-166" style="font-size: 70.7%; font-family: MathJax_Math-italic;">a</span><span class="mo" id="MathJax-Span-167" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-168" style="font-size: 70.7%; font-family: MathJax_Math-italic;">b</span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(2.769em, 1006.94em, 4.857em, -999.998em); top: -2.91em; left: 50%; margin-left: -3.444em;"><span class="msqrt" id="MathJax-Span-169"><span style="display: inline-block; position: relative; width: 6.944em; height: 0px;"><span style="position: absolute; clip: rect(3.012em, 1005.92em, 4.614em, -999.998em); top: -3.978em; left: 1.022em;"><span class="mrow" id="MathJax-Span-170"><span class="mo" id="MathJax-Span-171" style="font-family: MathJax_Size1; vertical-align: 0em;">∑</span><span class="msubsup" id="MathJax-Span-172" style="padding-left: 0.148em;"><span style="display: inline-block; position: relative; width: 1.847em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-173" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.352em, 1000.44em, 4.129em, -999.998em); top: -4.318em; left: 0.585em;"><span class="mn" id="MathJax-Span-174" style="font-size: 70.7%; font-family: MathJax_Main;">2</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.546em, 1001.26em, 4.274em, -999.998em); top: -3.833em; left: 0.585em;"><span class="texatom" id="MathJax-Span-175"><span class="mrow" id="MathJax-Span-176"><span class="mi" id="MathJax-Span-177" style="font-size: 70.7%; font-family: MathJax_Math-italic;">a</span><span class="mo" id="MathJax-Span-178" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-179" style="font-size: 70.7%; font-family: MathJax_Math-italic;">m</span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-180" style="font-family: MathJax_Size1; vertical-align: 0em; padding-left: 0.148em;">∑</span><span class="msubsup" id="MathJax-Span-181" style="padding-left: 0.148em;"><span style="display: inline-block; position: relative; width: 1.507em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-182" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.352em, 1000.44em, 4.129em, -999.998em); top: -4.318em; left: 0.585em;"><span class="mn" id="MathJax-Span-183" style="font-size: 70.7%; font-family: MathJax_Main;">2</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.352em, 1000.97em, 4.274em, -999.998em); top: -3.638em; left: 0.585em;"><span class="texatom" id="MathJax-Span-184"><span class="mrow" id="MathJax-Span-185"><span class="mi" id="MathJax-Span-186" style="font-size: 70.7%; font-family: MathJax_Math-italic;">a</span><span class="mo" id="MathJax-Span-187" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-188" style="font-size: 70.7%; font-family: MathJax_Math-italic;">b</span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.546em, 1005.92em, 3.886em, -999.998em); top: -4.755em; left: 1.022em;"><span style="display: inline-block; position: relative; width: 5.925em; height: 0px;"><span style="position: absolute; font-family: MathJax_Main; top: -3.978em; left: -0.095em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; font-family: MathJax_Main; top: -3.978em; left: 5.245em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 0.439em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 0.973em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 1.507em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 2.041em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 2.575em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 3.109em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 3.643em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 4.177em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 4.711em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(2.672em, 1001.02em, 4.76em, -999.998em); top: -3.93em; left: 0em;"><span style="font-family: MathJax_Size2;">√</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(0.876em, 1007.04em, 1.216em, -999.998em); top: -1.308em; left: 0em;"><span style="display: inline-block; overflow: hidden; vertical-align: 0em; border-top: 1.3px solid; width: 7.041em; height: 0px;"></span><span style="display: inline-block; width: 0px; height: 1.07em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.284em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.848em; border-left: 0px solid; width: 0px; height: 3.452em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML" display="block"><msubsup><mi>s</mi><mi>u</mi><mrow class="MJX-TeXAtom-ORD"><mi>c</mi><mi>o</mi><mi>s</mi></mrow></msubsup><mo stretchy="false">(</mo><msub><mi>i</mi><mi>m</mi></msub><mo>,</mo><msub><mi>i</mi><mi>b</mi></msub><mo stretchy="false">)</mo><mo>=</mo><mfrac><mrow><msub><mi>i</mi><mi>m</mi></msub><mo>⋅</mo><msub><mi>i</mi><mi>b</mi></msub></mrow><mrow><mrow><mo symmetric="true">‖</mo><msub><mi>i</mi><mi>m</mi></msub><mo symmetric="true">‖</mo></mrow><mrow><mo symmetric="true">‖</mo><msub><mi>i</mi><mi>b</mi></msub><mo symmetric="true">‖</mo></mrow></mrow></mfrac><mo>=</mo><mfrac><mrow><mo>∑</mo><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>a</mi><mo>,</mo><mi>m</mi></mrow></msub><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>a</mi><mo>,</mo><mi>b</mi></mrow></msub></mrow><msqrt><mo>∑</mo><msubsup><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>a</mi><mo>,</mo><mi>m</mi></mrow><mn>2</mn></msubsup><mo>∑</mo><msubsup><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>a</mi><mo>,</mo><mi>b</mi></mrow><mn>2</mn></msubsup></msqrt></mfrac></math></span></span></div><script type="math/tex; mode=display" id="MathJax-Element-10">
s_u^{cos}(i_m,i_b)
 = \frac{ i_m \cdot i_b }{ \left \| i_m \right \| \left \| i_b \right \| }
 = \frac{ \sum x_{a,m} x_{a,b} }{ \sqrt{ \sum x_{a,m}^2 \sum x_{a,b}^2 } }
</script></span></p>

İlk adımda kullanıcı-öğe matrisini oluşturmalıyız. Elimizde test ve eğitim olmak üzere 2 parça veri olduğu için iki matris oluşturmamız gerekecektir.

In [6]:
# Eğitim ve test için iki adet kullanıcı-öğe matrisi oluşturduk

train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]
    
test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

[Kosinüs benzerliğini](https://github.com/enesmanan/turkce-kitaplar/blob/main/Projelerle%20Yapay%20Zeka/Benzerlik_Algoritmalar%C4%B1/cosine_similarity.py) hesaplamak için sklearn kütüphanesindeki `pairwise_distances` fonksiyonunu kullanabiliriz.

Derecelendirmelerin tümü pozitif olduğundan çıktı 0 ile 1 arasında olacaktır.

In [7]:
from sklearn.metrics.pairwise import pairwise_distances

user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

Bir sonraki adım tahminler üretmektir. Benzerlik matrislerini: user_similarity ve item_similarity, kullanarak kullanıcı tabanlı CF için aşağıdaki formülü uygulayarak bir tahmin üretebiliriz.

<p><span class="math display"><span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" id="MathJax-Element-11-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot; display=&quot;block&quot;><msub><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mover><mi>x</mi><mo stretchy=&quot;false&quot;>&amp;#x005E;</mo></mover></mrow><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi>k</mi><mo>,</mo><mi>m</mi></mrow></msub><mo>=</mo><msub><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mover><mi>x</mi><mo stretchy=&quot;false&quot;>&amp;#x00AF;</mo></mover></mrow><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi>k</mi></mrow></msub><mo>+</mo><mfrac><mrow><munder><mo movablelimits=&quot;false&quot;>&amp;#x2211;</mo><mrow class=&quot;MJX-TeXAtom-ORD&quot;><msub><mi>u</mi><mi>a</mi></msub></mrow></munder><mi>s</mi><mi>i</mi><msub><mi>m</mi><mi>u</mi></msub><mo stretchy=&quot;false&quot;>(</mo><msub><mi>u</mi><mi>k</mi></msub><mo>,</mo><msub><mi>u</mi><mi>a</mi></msub><mo stretchy=&quot;false&quot;>)</mo><mo stretchy=&quot;false&quot;>(</mo><msub><mi>x</mi><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi>a</mi><mo>,</mo><mi>m</mi></mrow></msub><mo>&amp;#x2212;</mo><msub><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mover><mi>x</mi><mo stretchy=&quot;false&quot;>&amp;#x00AF;</mo></mover></mrow><mrow class=&quot;MJX-TeXAtom-ORD&quot;><msub><mi>u</mi><mi>a</mi></msub></mrow></msub><mo stretchy=&quot;false&quot;>)</mo></mrow><mrow><munder><mo movablelimits=&quot;false&quot;>&amp;#x2211;</mo><mrow class=&quot;MJX-TeXAtom-ORD&quot;><msub><mi>u</mi><mi>a</mi></msub></mrow></munder><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mo stretchy=&quot;false&quot;>|</mo></mrow><mi>s</mi><mi>i</mi><msub><mi>m</mi><mi>u</mi></msub><mo stretchy=&quot;false&quot;>(</mo><msub><mi>u</mi><mi>k</mi></msub><mo>,</mo><msub><mi>u</mi><mi>a</mi></msub><mo stretchy=&quot;false&quot;>)</mo><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mo stretchy=&quot;false&quot;>|</mo></mrow></mrow></mfrac></math>" role="presentation" style="text-align: center; position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-189" style="width: 18.206em; display: inline-block;"><span style="display: inline-block; position: relative; width: 17.672em; height: 0px; font-size: 103%;"><span style="position: absolute; clip: rect(-0.143em, 1017.67em, 4.226em, -999.998em); top: -2.279em; left: 0em;"><span class="mrow" id="MathJax-Span-190"><span class="msubsup" id="MathJax-Span-191"><span style="display: inline-block; position: relative; width: 1.847em; height: 0px;"><span style="position: absolute; clip: rect(3.109em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="texatom" id="MathJax-Span-192"><span class="mrow" id="MathJax-Span-193"><span class="munderover" id="MathJax-Span-194"><span style="display: inline-block; position: relative; width: 0.585em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-195" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1000.39em, 3.595em, -999.998em); top: -4.027em; left: 0.051em;"><span class="mo" id="MathJax-Span-196" style="font-family: MathJax_Main;">^</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.585em;"><span class="texatom" id="MathJax-Span-197"><span class="mrow" id="MathJax-Span-198"><span class="mi" id="MathJax-Span-199" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-200" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-201" style="font-size: 70.7%; font-family: MathJax_Math-italic;">m</span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-202" style="font-family: MathJax_Main; padding-left: 0.294em;">=</span><span class="msubsup" id="MathJax-Span-203" style="padding-left: 0.294em;"><span style="display: inline-block; position: relative; width: 1.022em; height: 0px;"><span style="position: absolute; clip: rect(3.206em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="texatom" id="MathJax-Span-204"><span class="mrow" id="MathJax-Span-205"><span class="munderover" id="MathJax-Span-206"><span style="display: inline-block; position: relative; width: 0.585em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-207" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.255em, 1000.44em, 3.595em, -999.998em); top: -3.978em; left: 0.051em;"><span class="mo" id="MathJax-Span-208" style="font-family: MathJax_Main;">¯</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.585em;"><span class="texatom" id="MathJax-Span-209"><span class="mrow" id="MathJax-Span-210"><span class="mi" id="MathJax-Span-211" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-212" style="font-family: MathJax_Main; padding-left: 0.245em;">+</span><span class="mfrac" id="MathJax-Span-213" style="padding-left: 0.245em;"><span style="display: inline-block; position: relative; width: 11.993em; height: 0px; margin-right: 0.1em; margin-left: 0.1em;"><span style="position: absolute; clip: rect(3.109em, 1011.8em, 5.197em, -999.998em); top: -5.483em; left: 50%; margin-left: -5.92em;"><span class="mrow" id="MathJax-Span-214"><span class="munderover" id="MathJax-Span-215"><span style="display: inline-block; position: relative; width: 1.07em; height: 0px;"><span style="position: absolute; clip: rect(3.109em, 1001.02em, 4.371em, -999.998em); top: -3.978em; left: 0em;"><span class="mo" id="MathJax-Span-216" style="font-family: MathJax_Size1; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.546em, 1000.73em, 4.323em, -999.998em); top: -3.104em; left: 0.148em;"><span class="texatom" id="MathJax-Span-217"><span class="mrow" id="MathJax-Span-218"><span class="msubsup" id="MathJax-Span-219"><span style="display: inline-block; position: relative; width: 0.731em; height: 0px;"><span style="position: absolute; clip: rect(3.546em, 1000.39em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-220" style="font-size: 70.7%; font-family: MathJax_Math-italic;">u</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.881em; left: 0.391em;"><span class="mi" id="MathJax-Span-221" style="font-size: 50%; font-family: MathJax_Math-italic;">a</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mi" id="MathJax-Span-222" style="font-family: MathJax_Math-italic; padding-left: 0.148em;">s</span><span class="mi" id="MathJax-Span-223" style="font-family: MathJax_Math-italic;">i</span><span class="msubsup" id="MathJax-Span-224"><span style="display: inline-block; position: relative; width: 1.362em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.88em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-225" style="font-family: MathJax_Math-italic;">m</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.876em;"><span class="mi" id="MathJax-Span-226" style="font-size: 70.7%; font-family: MathJax_Math-italic;">u</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-227" style="font-family: MathJax_Main;">(</span><span class="msubsup" id="MathJax-Span-228"><span style="display: inline-block; position: relative; width: 1.022em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-229" style="font-family: MathJax_Math-italic;">u</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.585em;"><span class="mi" id="MathJax-Span-230" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-231" style="font-family: MathJax_Main;">,</span><span class="msubsup" id="MathJax-Span-232" style="padding-left: 0.148em;"><span style="display: inline-block; position: relative; width: 1.022em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-233" style="font-family: MathJax_Math-italic;">u</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.585em;"><span class="mi" id="MathJax-Span-234" style="font-size: 70.7%; font-family: MathJax_Math-italic;">a</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-235" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-236" style="font-family: MathJax_Main;">(</span><span class="msubsup" id="MathJax-Span-237"><span style="display: inline-block; position: relative; width: 1.847em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-238" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.585em;"><span class="texatom" id="MathJax-Span-239"><span class="mrow" id="MathJax-Span-240"><span class="mi" id="MathJax-Span-241" style="font-size: 70.7%; font-family: MathJax_Math-italic;">a</span><span class="mo" id="MathJax-Span-242" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-243" style="font-size: 70.7%; font-family: MathJax_Math-italic;">m</span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-244" style="font-family: MathJax_Main; padding-left: 0.245em;">−</span><span class="msubsup" id="MathJax-Span-245" style="padding-left: 0.245em;"><span style="display: inline-block; position: relative; width: 1.362em; height: 0px;"><span style="position: absolute; clip: rect(3.206em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="texatom" id="MathJax-Span-246"><span class="mrow" id="MathJax-Span-247"><span class="munderover" id="MathJax-Span-248"><span style="display: inline-block; position: relative; width: 0.585em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-249" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.255em, 1000.44em, 3.595em, -999.998em); top: -3.978em; left: 0.051em;"><span class="mo" id="MathJax-Span-250" style="font-family: MathJax_Main;">¯</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.585em;"><span class="texatom" id="MathJax-Span-251"><span class="mrow" id="MathJax-Span-252"><span class="msubsup" id="MathJax-Span-253"><span style="display: inline-block; position: relative; width: 0.731em; height: 0px;"><span style="position: absolute; clip: rect(3.546em, 1000.39em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-254" style="font-size: 70.7%; font-family: MathJax_Math-italic;">u</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.881em; left: 0.391em;"><span class="mi" id="MathJax-Span-255" style="font-size: 50%; font-family: MathJax_Math-italic;">a</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-256" style="font-family: MathJax_Main;">)</span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.109em, 1007.04em, 5.197em, -999.998em); top: -3.25em; left: 50%; margin-left: -3.59em;"><span class="mrow" id="MathJax-Span-257"><span class="munderover" id="MathJax-Span-258"><span style="display: inline-block; position: relative; width: 1.07em; height: 0px;"><span style="position: absolute; clip: rect(3.109em, 1001.02em, 4.371em, -999.998em); top: -3.978em; left: 0em;"><span class="mo" id="MathJax-Span-259" style="font-family: MathJax_Size1; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.546em, 1000.73em, 4.323em, -999.998em); top: -3.104em; left: 0.148em;"><span class="texatom" id="MathJax-Span-260"><span class="mrow" id="MathJax-Span-261"><span class="msubsup" id="MathJax-Span-262"><span style="display: inline-block; position: relative; width: 0.731em; height: 0px;"><span style="position: absolute; clip: rect(3.546em, 1000.39em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-263" style="font-size: 70.7%; font-family: MathJax_Math-italic;">u</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.881em; left: 0.391em;"><span class="mi" id="MathJax-Span-264" style="font-size: 50%; font-family: MathJax_Math-italic;">a</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="texatom" id="MathJax-Span-265" style="padding-left: 0.148em;"><span class="mrow" id="MathJax-Span-266"><span class="mo" id="MathJax-Span-267" style="font-family: MathJax_Main;">|</span></span></span><span class="mi" id="MathJax-Span-268" style="font-family: MathJax_Math-italic;">s</span><span class="mi" id="MathJax-Span-269" style="font-family: MathJax_Math-italic;">i</span><span class="msubsup" id="MathJax-Span-270"><span style="display: inline-block; position: relative; width: 1.362em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.88em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-271" style="font-family: MathJax_Math-italic;">m</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.876em;"><span class="mi" id="MathJax-Span-272" style="font-size: 70.7%; font-family: MathJax_Math-italic;">u</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-273" style="font-family: MathJax_Main;">(</span><span class="msubsup" id="MathJax-Span-274"><span style="display: inline-block; position: relative; width: 1.022em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-275" style="font-family: MathJax_Math-italic;">u</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.585em;"><span class="mi" id="MathJax-Span-276" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-277" style="font-family: MathJax_Main;">,</span><span class="msubsup" id="MathJax-Span-278" style="padding-left: 0.148em;"><span style="display: inline-block; position: relative; width: 1.022em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-279" style="font-family: MathJax_Math-italic;">u</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.585em;"><span class="mi" id="MathJax-Span-280" style="font-size: 70.7%; font-family: MathJax_Math-italic;">a</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-281" style="font-family: MathJax_Main;">)</span><span class="texatom" id="MathJax-Span-282"><span class="mrow" id="MathJax-Span-283"><span class="mo" id="MathJax-Span-284" style="font-family: MathJax_Main;">|</span></span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(0.876em, 1011.99em, 1.216em, -999.998em); top: -1.308em; left: 0em;"><span style="display: inline-block; overflow: hidden; vertical-align: 0em; border-top: 1.3px solid; width: 11.993em; height: 0px;"></span><span style="display: inline-block; width: 0px; height: 1.07em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.284em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.898em; border-left: 0px solid; width: 0px; height: 4.252em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML" display="block"><msub><mrow class="MJX-TeXAtom-ORD"><mover><mi>x</mi><mo stretchy="false">^</mo></mover></mrow><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>m</mi></mrow></msub><mo>=</mo><msub><mrow class="MJX-TeXAtom-ORD"><mover><mi>x</mi><mo stretchy="false">¯</mo></mover></mrow><mrow class="MJX-TeXAtom-ORD"><mi>k</mi></mrow></msub><mo>+</mo><mfrac><mrow><munder><mo movablelimits="false">∑</mo><mrow class="MJX-TeXAtom-ORD"><msub><mi>u</mi><mi>a</mi></msub></mrow></munder><mi>s</mi><mi>i</mi><msub><mi>m</mi><mi>u</mi></msub><mo stretchy="false">(</mo><msub><mi>u</mi><mi>k</mi></msub><mo>,</mo><msub><mi>u</mi><mi>a</mi></msub><mo stretchy="false">)</mo><mo stretchy="false">(</mo><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>a</mi><mo>,</mo><mi>m</mi></mrow></msub><mo>−</mo><msub><mrow class="MJX-TeXAtom-ORD"><mover><mi>x</mi><mo stretchy="false">¯</mo></mover></mrow><mrow class="MJX-TeXAtom-ORD"><msub><mi>u</mi><mi>a</mi></msub></mrow></msub><mo stretchy="false">)</mo></mrow><mrow><munder><mo movablelimits="false">∑</mo><mrow class="MJX-TeXAtom-ORD"><msub><mi>u</mi><mi>a</mi></msub></mrow></munder><mrow class="MJX-TeXAtom-ORD"><mo stretchy="false">|</mo></mrow><mi>s</mi><mi>i</mi><msub><mi>m</mi><mi>u</mi></msub><mo stretchy="false">(</mo><msub><mi>u</mi><mi>k</mi></msub><mo>,</mo><msub><mi>u</mi><mi>a</mi></msub><mo stretchy="false">)</mo><mrow class="MJX-TeXAtom-ORD"><mo stretchy="false">|</mo></mrow></mrow></mfrac></math></span></span></div><script type="math/tex; mode=display" id="MathJax-Element-11">
\hat{x}_{k,m} = \bar{x}_{k} + \frac{\sum\limits_{u_a} sim_u(u_k, u_a) (x_{a,m} - \bar{x}_{u_a})}{\sum\limits_{u_a}|sim_u(u_k, u_a)|}
</script></span></p>

k ve a kullanıcıları arasındaki benzerliğe, benzer bir a kullanıcısının puanlarıyla (kullanıcının düzeltilmiş ortalama derecelendirmesi) çarpılan ağırlıklar olarak bakabiliriz. Derecelendirmelerin 1 ile 5 arasında kalması için normalleştirmemiz ve son adım olarak tahmin etmeye çalıştığımız kullanıcı için ortalama derecelendirmeleri toplamamız gerekecektir.

Buradaki fikir şudur: Bazı kullanıcılar, tüm filmlere yüksek veya düşük puan vermeye eğilimli olabilir. Bu kullanıcıların verdikleri puanların mutlak değerlerinden ziyade, aralarındaki göreceli fark daha önemlidir. Yani, bir kullanıcının bir filme 5 vermesi, o filmi çok sevdiği anlamına gelmeyebilir. Belki de o kullanıcı, beğendiği tüm filmlere 5 veriyor ve beğenmediği filmlere 4 veriyordur. Bu durumda, o kullanıcının puanları, filmleri ne kadar sevdiğini tam olarak yansıtmaz.

Öğe tabanlı işbirlikçi filtreleme için tahmin yaparken, sorgulanan kullanıcının kendisi tahmin yaparken kullanıldığı için kullanıcıların ortalama puanlarınıın düzeltilmesine gerek yoktur.

<p><span class="math display"><span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" id="MathJax-Element-17-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot; display=&quot;block&quot;><msub><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mover><mi>x</mi><mo stretchy=&quot;false&quot;>&amp;#x005E;</mo></mover></mrow><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi>k</mi><mo>,</mo><mi>m</mi></mrow></msub><mo>=</mo><mfrac><mrow><munder><mo movablelimits=&quot;false&quot;>&amp;#x2211;</mo><mrow class=&quot;MJX-TeXAtom-ORD&quot;><msub><mi>i</mi><mi>b</mi></msub></mrow></munder><mi>s</mi><mi>i</mi><msub><mi>m</mi><mi>i</mi></msub><mo stretchy=&quot;false&quot;>(</mo><msub><mi>i</mi><mi>m</mi></msub><mo>,</mo><msub><mi>i</mi><mi>b</mi></msub><mo stretchy=&quot;false&quot;>)</mo><mo stretchy=&quot;false&quot;>(</mo><msub><mi>x</mi><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi>k</mi><mo>,</mo><mi>b</mi></mrow></msub><mo stretchy=&quot;false&quot;>)</mo></mrow><mrow><munder><mo movablelimits=&quot;false&quot;>&amp;#x2211;</mo><mrow class=&quot;MJX-TeXAtom-ORD&quot;><msub><mi>i</mi><mi>b</mi></msub></mrow></munder><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mo stretchy=&quot;false&quot;>|</mo></mrow><mi>s</mi><mi>i</mi><msub><mi>m</mi><mi>i</mi></msub><mo stretchy=&quot;false&quot;>(</mo><msub><mi>i</mi><mi>m</mi></msub><mo>,</mo><msub><mi>i</mi><mi>b</mi></msub><mo stretchy=&quot;false&quot;>)</mo><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mo stretchy=&quot;false&quot;>|</mo></mrow></mrow></mfrac></math>" role="presentation" style="text-align: center; position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-300" style="width: 12.381em; display: inline-block;"><span style="display: inline-block; position: relative; width: 11.993em; height: 0px; font-size: 103%;"><span style="position: absolute; clip: rect(-0.192em, 1011.99em, 4.226em, -999.998em); top: -2.279em; left: 0em;"><span class="mrow" id="MathJax-Span-301"><span class="msubsup" id="MathJax-Span-302"><span style="display: inline-block; position: relative; width: 1.847em; height: 0px;"><span style="position: absolute; clip: rect(3.109em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="texatom" id="MathJax-Span-303"><span class="mrow" id="MathJax-Span-304"><span class="munderover" id="MathJax-Span-305"><span style="display: inline-block; position: relative; width: 0.585em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-306" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1000.39em, 3.595em, -999.998em); top: -4.027em; left: 0.051em;"><span class="mo" id="MathJax-Span-307" style="font-family: MathJax_Main;">^</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.585em;"><span class="texatom" id="MathJax-Span-308"><span class="mrow" id="MathJax-Span-309"><span class="mi" id="MathJax-Span-310" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-311" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-312" style="font-size: 70.7%; font-family: MathJax_Math-italic;">m</span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-313" style="font-family: MathJax_Main; padding-left: 0.294em;">=</span><span class="mfrac" id="MathJax-Span-314" style="padding-left: 0.294em;"><span style="display: inline-block; position: relative; width: 8.595em; height: 0px; margin-right: 0.1em; margin-left: 0.1em;"><span style="position: absolute; clip: rect(3.109em, 1008.4em, 5.245em, -999.998em); top: -5.532em; left: 50%; margin-left: -4.221em;"><span class="mrow" id="MathJax-Span-315"><span class="munderover" id="MathJax-Span-316"><span style="display: inline-block; position: relative; width: 1.07em; height: 0px;"><span style="position: absolute; clip: rect(3.109em, 1001.02em, 4.371em, -999.998em); top: -3.978em; left: 0em;"><span class="mo" id="MathJax-Span-317" style="font-family: MathJax_Size1; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.352em, 1000.54em, 4.323em, -999.998em); top: -3.104em; left: 0.294em;"><span class="texatom" id="MathJax-Span-318"><span class="mrow" id="MathJax-Span-319"><span class="msubsup" id="MathJax-Span-320"><span style="display: inline-block; position: relative; width: 0.536em; height: 0px;"><span style="position: absolute; clip: rect(3.352em, 1000.2em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-321" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.881em; left: 0.245em;"><span class="mi" id="MathJax-Span-322" style="font-size: 50%; font-family: MathJax_Math-italic;">b</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mi" id="MathJax-Span-323" style="font-family: MathJax_Math-italic; padding-left: 0.148em;">s</span><span class="mi" id="MathJax-Span-324" style="font-family: MathJax_Math-italic;">i</span><span class="msubsup" id="MathJax-Span-325"><span style="display: inline-block; position: relative; width: 1.216em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.88em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-326" style="font-family: MathJax_Math-italic;">m</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.876em;"><span class="mi" id="MathJax-Span-327" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-328" style="font-family: MathJax_Main;">(</span><span class="msubsup" id="MathJax-Span-329"><span style="display: inline-block; position: relative; width: 1.022em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.29em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-330" style="font-family: MathJax_Math-italic;">i</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.342em;"><span class="mi" id="MathJax-Span-331" style="font-size: 70.7%; font-family: MathJax_Math-italic;">m</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-332" style="font-family: MathJax_Main;">,</span><span class="msubsup" id="MathJax-Span-333" style="padding-left: 0.148em;"><span style="display: inline-block; position: relative; width: 0.731em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.29em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-334" style="font-family: MathJax_Math-italic;">i</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.342em;"><span class="mi" id="MathJax-Span-335" style="font-size: 70.7%; font-family: MathJax_Math-italic;">b</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-336" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-337" style="font-family: MathJax_Main;">(</span><span class="msubsup" id="MathJax-Span-338"><span style="display: inline-block; position: relative; width: 1.507em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-339" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.585em;"><span class="texatom" id="MathJax-Span-340"><span class="mrow" id="MathJax-Span-341"><span class="mi" id="MathJax-Span-342" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-343" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-344" style="font-size: 70.7%; font-family: MathJax_Math-italic;">b</span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-345" style="font-family: MathJax_Main;">)</span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.109em, 1006.6em, 5.245em, -999.998em); top: -3.25em; left: 50%; margin-left: -3.347em;"><span class="mrow" id="MathJax-Span-346"><span class="munderover" id="MathJax-Span-347"><span style="display: inline-block; position: relative; width: 1.07em; height: 0px;"><span style="position: absolute; clip: rect(3.109em, 1001.02em, 4.371em, -999.998em); top: -3.978em; left: 0em;"><span class="mo" id="MathJax-Span-348" style="font-family: MathJax_Size1; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.352em, 1000.54em, 4.323em, -999.998em); top: -3.104em; left: 0.294em;"><span class="texatom" id="MathJax-Span-349"><span class="mrow" id="MathJax-Span-350"><span class="msubsup" id="MathJax-Span-351"><span style="display: inline-block; position: relative; width: 0.536em; height: 0px;"><span style="position: absolute; clip: rect(3.352em, 1000.2em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-352" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.881em; left: 0.245em;"><span class="mi" id="MathJax-Span-353" style="font-size: 50%; font-family: MathJax_Math-italic;">b</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="texatom" id="MathJax-Span-354" style="padding-left: 0.148em;"><span class="mrow" id="MathJax-Span-355"><span class="mo" id="MathJax-Span-356" style="font-family: MathJax_Main;">|</span></span></span><span class="mi" id="MathJax-Span-357" style="font-family: MathJax_Math-italic;">s</span><span class="mi" id="MathJax-Span-358" style="font-family: MathJax_Math-italic;">i</span><span class="msubsup" id="MathJax-Span-359"><span style="display: inline-block; position: relative; width: 1.216em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.88em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-360" style="font-family: MathJax_Math-italic;">m</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.876em;"><span class="mi" id="MathJax-Span-361" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-362" style="font-family: MathJax_Main;">(</span><span class="msubsup" id="MathJax-Span-363"><span style="display: inline-block; position: relative; width: 1.022em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.29em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-364" style="font-family: MathJax_Math-italic;">i</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.342em;"><span class="mi" id="MathJax-Span-365" style="font-size: 70.7%; font-family: MathJax_Math-italic;">m</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-366" style="font-family: MathJax_Main;">,</span><span class="msubsup" id="MathJax-Span-367" style="padding-left: 0.148em;"><span style="display: inline-block; position: relative; width: 0.731em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.29em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-368" style="font-family: MathJax_Math-italic;">i</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.342em;"><span class="mi" id="MathJax-Span-369" style="font-size: 70.7%; font-family: MathJax_Math-italic;">b</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-370" style="font-family: MathJax_Main;">)</span><span class="texatom" id="MathJax-Span-371"><span class="mrow" id="MathJax-Span-372"><span class="mo" id="MathJax-Span-373" style="font-family: MathJax_Main;">|</span></span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(0.876em, 1008.6em, 1.216em, -999.998em); top: -1.308em; left: 0em;"><span style="display: inline-block; overflow: hidden; vertical-align: 0em; border-top: 1.3px solid; width: 8.595em; height: 0px;"></span><span style="display: inline-block; width: 0px; height: 1.07em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.284em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.898em; border-left: 0px solid; width: 0px; height: 4.353em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML" display="block"><msub><mrow class="MJX-TeXAtom-ORD"><mover><mi>x</mi><mo stretchy="false">^</mo></mover></mrow><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>m</mi></mrow></msub><mo>=</mo><mfrac><mrow><munder><mo movablelimits="false">∑</mo><mrow class="MJX-TeXAtom-ORD"><msub><mi>i</mi><mi>b</mi></msub></mrow></munder><mi>s</mi><mi>i</mi><msub><mi>m</mi><mi>i</mi></msub><mo stretchy="false">(</mo><msub><mi>i</mi><mi>m</mi></msub><mo>,</mo><msub><mi>i</mi><mi>b</mi></msub><mo stretchy="false">)</mo><mo stretchy="false">(</mo><msub><mi>x</mi><mrow class="MJX-TeXAtom-ORD"><mi>k</mi><mo>,</mo><mi>b</mi></mrow></msub><mo stretchy="false">)</mo></mrow><mrow><munder><mo movablelimits="false">∑</mo><mrow class="MJX-TeXAtom-ORD"><msub><mi>i</mi><mi>b</mi></msub></mrow></munder><mrow class="MJX-TeXAtom-ORD"><mo stretchy="false">|</mo></mrow><mi>s</mi><mi>i</mi><msub><mi>m</mi><mi>i</mi></msub><mo stretchy="false">(</mo><msub><mi>i</mi><mi>m</mi></msub><mo>,</mo><msub><mi>i</mi><mi>b</mi></msub><mo stretchy="false">)</mo><mrow class="MJX-TeXAtom-ORD"><mo stretchy="false">|</mo></mrow></mrow></mfrac></math></span></span></div><script type="math/tex; mode=display" id="MathJax-Element-17">
\hat{x}_{k,m} = \frac{\sum\limits_{i_b} sim_i(i_m, i_b) (x_{k,b}) }{\sum\limits_{i_b}|sim_i(i_m, i_b)|}
</script></span></p>

In [8]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

### Değerlendirme
Tahmin edilen derecelendirmelerin doğruluğunu değerlendirmek için bir çok değerlendirme metriği bulunsa da kullanılan en popüler metriklerden biri <b>*Hata Kareler Ortalamasının Karekökü* (Root Mean Squared Error - RMSE)</b>'dür.

<p><span class="math display"><span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" id="MathJax-Element-18-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot; display=&quot;block&quot;><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi class=&quot;MJX-tex-mathit&quot; mathvariant=&quot;italic&quot;>R</mi><mi class=&quot;MJX-tex-mathit&quot; mathvariant=&quot;italic&quot;>M</mi><mi class=&quot;MJX-tex-mathit&quot; mathvariant=&quot;italic&quot;>S</mi><mi class=&quot;MJX-tex-mathit&quot; mathvariant=&quot;italic&quot;>E</mi></mrow><mo>=</mo><msqrt><mfrac><mn>1</mn><mi>N</mi></mfrac><mo>&amp;#x2211;</mo><mo stretchy=&quot;false&quot;>(</mo><msub><mi>x</mi><mi>i</mi></msub><mo>&amp;#x2212;</mo><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mover><msub><mi>x</mi><mi>i</mi></msub><mo stretchy=&quot;false&quot;>&amp;#x005E;</mo></mover></mrow><msup><mo stretchy=&quot;false&quot;>)</mo><mn>2</mn></msup></msqrt></math>" role="presentation" style="text-align: center; position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-374" style="width: 12.721em; display: inline-block;"><span style="display: inline-block; position: relative; width: 12.333em; height: 0px; font-size: 103%;"><span style="position: absolute; clip: rect(0.633em, 1012.33em, 3.303em, -999.998em); top: -2.279em; left: 0em;"><span class="mrow" id="MathJax-Span-375"><span class="texatom" id="MathJax-Span-376"><span class="mrow" id="MathJax-Span-377"><span class="mi" id="MathJax-Span-378" style="font-family: MathJax_Main-italic;">R</span><span class="mi" id="MathJax-Span-379" style="font-family: MathJax_Main-italic;">M</span><span class="mi" id="MathJax-Span-380" style="font-family: MathJax_Main-italic;">S</span><span class="mi" id="MathJax-Span-381" style="font-family: MathJax_Main-italic;">E</span></span></span><span class="mo" id="MathJax-Span-382" style="font-family: MathJax_Main; padding-left: 0.294em;">=</span><span class="msqrt" id="MathJax-Span-383" style="padding-left: 0.294em;"><span style="display: inline-block; position: relative; width: 8.109em; height: 0px;"><span style="position: absolute; clip: rect(2.478em, 1007.09em, 4.808em, -999.998em); top: -3.978em; left: 1.022em;"><span class="mrow" id="MathJax-Span-384"><span class="mfrac" id="MathJax-Span-385"><span style="display: inline-block; position: relative; width: 1.022em; height: 0px; margin-right: 0.1em; margin-left: 0.1em;"><span style="position: absolute; clip: rect(3.158em, 1000.44em, 4.129em, -999.998em); top: -4.658em; left: 50%; margin-left: -0.24em;"><span class="mn" id="MathJax-Span-386" style="font-family: MathJax_Main;">1</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1000.88em, 4.129em, -999.998em); top: -3.299em; left: 50%; margin-left: -0.434em;"><span class="mi" id="MathJax-Span-387" style="font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.1em;"></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(0.876em, 1001.02em, 1.216em, -999.998em); top: -1.308em; left: 0em;"><span style="display: inline-block; overflow: hidden; vertical-align: 0em; border-top: 1.3px solid; width: 1.022em; height: 0px;"></span><span style="display: inline-block; width: 0px; height: 1.07em;"></span></span></span></span><span class="mo" id="MathJax-Span-388" style="font-family: MathJax_Size2; vertical-align: 0em; padding-left: 0.148em;">∑</span><span class="mo" id="MathJax-Span-389" style="font-family: MathJax_Main;">(</span><span class="msubsup" id="MathJax-Span-390"><span style="display: inline-block; position: relative; width: 0.876em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-391" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.585em;"><span class="mi" id="MathJax-Span-392" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span class="mo" id="MathJax-Span-393" style="font-family: MathJax_Main; padding-left: 0.245em;">−</span><span class="texatom" id="MathJax-Span-394" style="padding-left: 0.245em;"><span class="mrow" id="MathJax-Span-395"><span class="munderover" id="MathJax-Span-396"><span style="display: inline-block; position: relative; width: 0.876em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.88em, 4.274em, -999.998em); top: -3.978em; left: 0em;"><span class="msubsup" id="MathJax-Span-397"><span style="display: inline-block; position: relative; width: 0.876em; height: 0px;"><span style="position: absolute; clip: rect(3.4em, 1000.54em, 4.129em, -999.998em); top: -3.978em; left: 0em;"><span class="mi" id="MathJax-Span-398" style="font-family: MathJax_Math-italic;">x</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -3.833em; left: 0.585em;"><span class="mi" id="MathJax-Span-399" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1000.39em, 3.595em, -999.998em); top: -4.027em; left: 0.197em;"><span class="mo" id="MathJax-Span-400" style="font-family: MathJax_Main;">^</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span></span></span><span class="msubsup" id="MathJax-Span-401"><span style="display: inline-block; position: relative; width: 0.828em; height: 0px;"><span style="position: absolute; clip: rect(3.109em, 1000.29em, 4.371em, -999.998em); top: -3.978em; left: 0em;"><span class="mo" id="MathJax-Span-402" style="font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; top: -4.415em; left: 0.391em;"><span class="mn" id="MathJax-Span-403" style="font-size: 70.7%; font-family: MathJax_Main;">2</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(3.546em, 1007.14em, 3.886em, -999.998em); top: -5.192em; left: 1.022em;"><span style="display: inline-block; position: relative; width: 7.138em; height: 0px;"><span style="position: absolute; font-family: MathJax_Main; top: -3.978em; left: -0.095em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; font-family: MathJax_Main; top: -3.978em; left: 6.41em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 0.439em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 0.973em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 1.507em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 2.09em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 2.624em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 3.158em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 3.692em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 4.274em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 4.808em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 5.342em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="font-family: MathJax_Main; position: absolute; top: -3.978em; left: 5.876em;">−<span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span><span style="position: absolute; clip: rect(2.381em, 1001.02em, 5.1em, -999.998em); top: -4.027em; left: 0em;"><span style="font-family: MathJax_Size3;">√</span><span style="display: inline-block; width: 0px; height: 3.983em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.284em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.948em; border-left: 0px solid; width: 0px; height: 2.552em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML" display="block"><mrow class="MJX-TeXAtom-ORD"><mi class="MJX-tex-mathit" mathvariant="italic">R</mi><mi class="MJX-tex-mathit" mathvariant="italic">M</mi><mi class="MJX-tex-mathit" mathvariant="italic">S</mi><mi class="MJX-tex-mathit" mathvariant="italic">E</mi></mrow><mo>=</mo><msqrt><mfrac><mn>1</mn><mi>N</mi></mfrac><mo>∑</mo><mo stretchy="false">(</mo><msub><mi>x</mi><mi>i</mi></msub><mo>−</mo><mrow class="MJX-TeXAtom-ORD"><mover><msub><mi>x</mi><mi>i</mi></msub><mo stretchy="false">^</mo></mover></mrow><msup><mo stretchy="false">)</mo><mn>2</mn></msup></msqrt></math></span></span></div><script type="math/tex; mode=display" id="MathJax-Element-18">
\mathit{RMSE} =\sqrt{\frac{1}{N} \sum (x_i -\hat{x_i})^2}
</script></span></p>

RMSE hesaplamak için Sklearn içerisinden metrics altında bulunan mean_square_error (MSE) fonksiyonunu kullanabiliriz, tek fark RMSE'ye ulaşmak için  MSE değerinin karekökünü almamız gerekir.

In [9]:
from math import sqrt
from sklearn.metrics import mean_squared_error

def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 3.115702902420396
Item-based CF RMSE: 3.441244284152521


Bellek tabanlı CF algoritmalarının avantajları; uygulaması kolaydır ve tatmin edici sonuçlar üretir. Dezavantajları, gerçek dünya senaryolarında yeterince iyi ölçeklenemiyor oluşu ve tavsiye sistemlerinin en popüler problemi olan <b>*soğuk başlangıç (cold start)*</b> sorununa, yani yeni bir kullanıcı veya yeni öğenin sisteme girilmesi durumunda sistemde yeni öğe/kullanıcı ile ilgili herhangi bir veri bulunmaması kaynaklı sistemin sağlıklı önerilerde bulunamama durumuna çözüm sunamaması. Model tabanlı CF yöntemleri ise ölçeklenebilir ve bellek tabanlı modellerden daha yüksek seyreklik derecesiyle başa çıkabilir, ancak soğuk başlangıç probleminde model tabanlı algoritma da sorun yaşar. 

### Model tabanlı İşbirlikçi Filtreleme

Model tabanlı İşbirlikçi Filtreleme, özellikle latent (gizli/gizil) değişken ayrıştırma ve boyut indirgeme için kullanılan, çoğunlukla denetimsiz öğrenme yöntemi olarak bilinen <b>*matris faktörizasyonuna (MF)*</b> dayanmaktadır. Matris faktörizasyonu, genellikle Bellek tabanlı CF'e göre daha iyi ölçeklenebilir ve seyreklikle başa çıkabilir. MF'nin amacı, bilinen derecelendirmelerden kullanıcıların latent tercihlerini ve ürünlerin latent özniteliklerini öğrenmek (derecelendirmelerin özelliklerini açıklayan özellikleri öğrenmek) ve ardından bilinmeyen derecelendirmeleri kullanıcıların ve ürünlerin latent özelliklerinin nokta çarpımı yoluyla tahmin etmektir. Çok seyrek bir matris ve birçok boyut olduğunda, matris faktörizasyonu yaparak kullanıcı-ürün matrisini düşük-rütbeli (low-rank) bir yapıya yeniden yapılandırabiliriz ve matrisi, iki düşük-rütbeli matrisin çarpımıyla temsil edebiliriz, burada satırlar latent vektörleri içerir.

MovieLens veri seti için seyrekliğe bakalım:

In [10]:
sparsity = round(1.0-len(df)/float(n_users*n_items),3)
print('The sparsity level of MovieLens100K is ' +  str(sparsity*100) + '%')

The sparsity level of MovieLens100K is 93.7%


Kullanıcıların ve öğelerin öğrenilen latent tercihlerine bir örnek vermek gerekirse: Diyelim ki MovieLens veri kümesi için bazı bilgilere sahibiz:(user id, age, location, gender, movie id, director, actor, language, year, rating). Matris faktörizasyonunu kullanarak model, kullanıcıların önemli özelliklerinin yaş grubu (10 yaş altı, 10–18, 18–30, 30–90), konum ve cinsiyet olduğunu öğrenir. Ayrıca, filmlerin önemli özelliklerinin decade aralığı, yönetmen ve aktör olduğunu öğrenir. Şimdi, elimizde bulunan verilere baktığımızda, "decate" gibi bir özellik olmadığını görebiliriz, buradaki durum model bu özelliği kendiliğinden öğrenmiştir. Önemli olan şudur: İşbirlikçi Filtreleme, latent özellikleri öğrenmek için sadece verileri (kullanıcı_id, film_id, derecelendirme) kullanır. Eğer sınırlı veri mevcutsa, model tabanlı IF modeli kötü tahminler yapabilir çünkü latent özellikleri öğrenmek daha zor olacaktır.

Derecelendirmeleri ve içerik özelliklerini bir arada kullanan modellere, İşbirlikçi ve İçerik Tabanlı Modellerin birleştirildiği <b>*"Hybrid Öneri Sistemleri"*</b> denir. Genellikle, hybrid öneri sistemleri İşbirlikçi veya İçerik Tabanlı Modellerin tek başlarına gösterdiği doğruluk seviyesinin üzerinde bir performans sergilerler. Ayrıca bu sistemler, soğuk başlangıç sorununu daha iyi ele alabilirler çünkü bir kullanıcının veya bir öğenin herhangi bir derecelendirmesi yoksa, kullanıcı veya öğe metaverilerini kullanarak tahminlerde bulunabilirler.

### SVD

İyi bilinen bir matris faktörizasyon yöntemi, Tekil değer ayrıştırmasıdır (Singular Value Decomposition - SVD). İşbirlikçi Filtreleme, tekil değer ayrışımı kullanılarak bir $X$ matrisine yaklaşılarak formüle edilebilir. Netflix Prize yarışmasında kazanan ekip, ürün önerileri oluşturmak için SVD matris faktörizasyon modellerini kullandı. Genel denklem şu şekilde ifade edilebilir: $X=UxSxV^T$

Bir $mxn$ boyutunda $X$ matrisi verildiğinde:

+ U bir $mxr$ ortogonal matristir
+ S köşegeninde pozitif olmayan gerçek sayılar bulunan $rxr$ boyutlu bir diyagonal matristir.
+ VT bir $rxn$ ortogonal matristir
+ S matrisindeki köşegen üzerindeki elemanlar, X matrisinin tekil değerleridir.

Matris $X$, $U$, $S$ ve $V$ olarak ayrıştırılabilir. $U$ matrisi, gizli özellik uzayındaki kullanıcılara karşılık gelen özellik vektörlerini temsil ederken, $V$ matrisi, gizli özellik uzayındaki öğelere karşılık gelen özellik vektörlerini temsil eder.

Şimdi, tahminler yapmak için $U$, $S$ ve $V^T$ matrislerinin noktasal çarpımını alarak işlem yapabiliriz. Tahminleri aşağıdaki şekilde hesaplayabiliriz:

In [11]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

u, s, vt = svds(train_data_matrix, k = 20)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)

print('User-based CF MSE: ' + str(rmse(X_pred, test_data_matrix)))

User-based CF MSE: 2.714235669194119


Özetle:

Bu yazıda, basit İşbirlikçi Filtreleme yöntemlerini, hem bellek tabanlı CF hem de model tabanlı CF'i nasıl uygulanabileceğini ele aldık.
+ Bellek tabanlı modeller, öğeler veya kullanıcılar arasındaki benzerlik temeline dayanır, burada kosinüs benzerliğini kullanırız.
+ Model tabanlı CF, matris faktörizasyona dayalıdır ve matrisi ayrıştırmak için SVD kullanırız.
Soğuk başlangıç senaryolarında (yeni kullanıcılar ve öğeler için az verinin bulunduğu durumlar) iyi performans gösteren öneri sistemleri oluşturmak hala bir zorluktur. Standart işbirlikçi filtreleme yöntemi bu tür durumlarda kötü performans gösterebilir.

Okuduğunuz için teşekkürler. İçeriğin büyük bir kısmı [buradaki](https://www.kaggle.com/code/ashishpatel26/movie-recommendation-of-movie-lens-data-set/notebook#1.Item-base-movie-Recommendation-System) notebooktan çevrilmiştir.

Kaynaklar:

+ [https://www.ethanrosenthal.com/2015/11/02/intro-to-collaborative-filtering/](https://www.ethanrosenthal.com/2015/11/02/intro-to-collaborative-filtering/)

+ [https://buomsoo-kim.github.io/recommender%20systems/2020/09/25/Recommender-systems-collab-filtering-12.md/](https://buomsoo-kim.github.io/recommender%20systems/2020/09/25/Recommender-systems-collab-filtering-12.md/)

+ [https://www.kaggle.com/code/ashishpatel26/movie-recommendation-of-movie-lens-data-set/notebook#1.Item-base-movie-Recommendation-System](https://www.kaggle.com/code/ashishpatel26/movie-recommendation-of-movie-lens-data-set/notebook#1.Item-base-movie-Recommendation-System)


+ [https://github.com/rare-technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb](https://github.com/rare-technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb)